<a href="https://colab.research.google.com/github/ZainaHweij/csdiagnostic/blob/main/TinyML_ACS_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Train TensorFlow Lite regression algorithm

In [ ]:
# Train Regression Algorithm

# Import Libraries
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load dataset fsr_data.csv
df = pd.read_csv('/content/drive/My Drive/SEF 2023-2024/TinyML/fsr_data.csv')

# Prepare data for training
scaler = MinMaxScaler()
df[['fsr_1', 'fsr_2','fsr_3','fsr_4','fsr_5','fsr_6','temp','humid','gyro', 'ground_truth']] = scaler.fit_transform(df[['fsr_1', 'fsr_2','fsr_3','fsr_4','fsr_5','fsr_6','temp','humid','gyro', 'ground_truth']])

# Split data into features (X) and target (y)
X = df[['fsr_1', 'fsr_2','fsr_3','fsr_4','fsr_5','fsr_6','temp','humid','gyro']]
y = df['ground_truth']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(9,)),  # Assuming 9 features (sensor readings)
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Train the Model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('/content/drive/My Drive/SEF 2023-2024/TinyML/model.tflite', 'wb') as f:
    f.write(tflite_model)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/10
3/3 [==============================] - 1s 75ms/step - loss: 0.3344 - val_loss: 0.2171
Epoch 2/10
3/3 [==============================] - 0s 18ms/step - loss: 0.2409 - val_loss: 0.1534
Epoch 3/10
3/3 [==============================] - 0s 27ms/step - loss: 0.1598 - val_loss: 0.1044
Epoch 4/10
3/3 [==============================] - 0s 16ms/step - loss: 0.1001 - val_loss: 0.0685
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 0.0585 - val_loss: 0.0442
Epoch 6/10
3/3 [==============================] - 0s 17ms/step - loss: 0.0312 - val_loss: 0.0303
Epoch 7/10
3/3 [==============================] - 0s 19ms/step - loss: 0.0171 - val_loss: 0.0244
Epoch 8/10
3/3 [==============================] - 0s 17ms/step - loss: 0.0128 - val_loss: 0.0234
Epoch 9/10
3/3 [==============================] - 0s 17ms/step - loss: 0.0151 - val_loss: 0.024

Save the TFLite model in .h format (to work with Arduino Sketch)

In [ ]:
tflite_model_path = '/content/drive/My Drive/SEF 2023-2024/TinyML/model.h'
with open(tflite_model_path, 'w') as f:  # Open the file in text mode
    f.write('#ifndef MODEL_H\n')
    f.write('#define MODEL_H\n\n')
    f.write('#include <cstdint>\n\n')
    f.write('const uint8_t model[] = {\n')
    for byte in tflite_model:
        f.write(hex(byte) + ', ')
    f.write('\n};\n\n#endif\n')

# Print the path for reference
print(f'TFLite model saved as {tflite_model_path}')

TFLite model saved as /content/drive/My Drive/SEF 2023-2024/TinyML/model.h


### Model Performance

In [ ]:
# Model regression statistics

# Evaluate the Model
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

# Predictions on Test Set
y_pred = model.predict(X_test)

# Calculate additional performance metrics
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error on Test Set: {mae}')
print(f'R-squared on Test Set: {r2}')

1/1 [==============================] - 0s 24ms/step - loss: 0.0252
Mean Squared Error on Test Set: 0.025173742324113846
1/1 [==============================] - 0s 20ms/step
Mean Absolute Error on Test Set: 0.130887714301998
R-squared on Test Set: 0.7202686082075032


In [ ]:
from sklearn.metrics import precision_score, accuracy_score, confusion_matrix

# Set a threshold for classification
threshold = 0.5

# Convert regression predictions to binary classification
y_pred_binary = (y_pred >= threshold).astype(int)
y_test_binary = (y_test >= threshold).astype(int)

# Calculate precision, accuracy, specificity, and recall
precision = precision_score(y_test_binary, y_pred_binary)
accuracy = accuracy_score(y_test_binary, y_pred_binary)

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test_binary, y_pred_binary).ravel()

# Calculate specificity and recall
specificity = tn / (tn + fp)
recall = tp / (tp + fn)

print(f'Precision: {precision}')
print(f'Accuracy: {accuracy}')
print(f'Specificity: {specificity}')
print(f'Recall: {recall}')

Precision: 0.9
Accuracy: 0.95
Specificity: 0.9090909090909091
Recall: 1.0


### Manually test trained model

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TensorFlow Lite model
model_path = '/content/drive/My Drive/SEF 2023-2024/TinyML/model.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Define a function to get input from the user
def get_user_input():
    # Ask the user for input values
    fsr_1 = float(input("Enter FSR 1 value: "))
    fsr_2 = float(input("Enter FSR 2 value: "))
    fsr_3 = float(input("Enter FSR 3 value: "))
    fsr_4 = float(input("Enter FSR 4 value: "))
    fsr_5 = float(input("Enter FSR 5 value: "))
    fsr_6 = float(input("Enter FSR 6 value: "))
    temp = float(input("Enter temperature value: "))
    humid = float(input("Enter humidity value: "))
    gyro = float(input("Enter gyro value: "))

    # Create a numpy array with the user input
    user_input = np.array([fsr_1, fsr_2, fsr_3, fsr_4, fsr_5, fsr_6, temp, humid, gyro], dtype=np.float32).reshape(1, 9)
    return user_input

# Get user input
processed_data = get_user_input()

# Set input tensor
input_details = interpreter.get_input_details()
interpreter.set_tensor(input_details[0]['index'], processed_data)

# Run inference
interpreter.invoke()

# Get output tensor
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])

# Process the output data as needed
print("Processed Input Data:", processed_data)
print("Output Data:", output_data)

Enter FSR 1 value: 30
Enter FSR 2 value: 30
Enter FSR 3 value: 31
Enter FSR 4 value: 30
Enter FSR 5 value: 30
Enter FSR 6 value: 30
Enter temperature value: 37
Enter humidity value: 50
Enter gyro value: 0
Processed Input Data: [[30. 30. 31. 30. 30. 30. 37. 50.  0.]]
Output Data: [[35.8698]]
